### Youtube comments analysis
In this notebook, we have a dataset of user comments for youtube videos related to animals or pets. We will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.

The dataset provided for this coding test are comments for videos related to animals and/or pets. The dataset is 240MB compressed
https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

 The dataset file is comma separated, with a header line defining the field names, listed here: <br>
● creator_name. Name of the YouTube channel creator.<br>
● userid. Integer identifier for the users commenting on the YouTube channels.<br>
● comment. Text of the comments made by the users.

#### Download the data

In [4]:
# link: https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a', dest_path='./../../dbfs/Youtube/data/animal_comments.gz')


# 0. Data Exploration and Cleaning

In [6]:
# read data
df = spark.read.load('/Youtube/data/animal_comments.gz', format='csv', header = True, inferSchema = True)
df.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [7]:
df.count() 

Out[9]: 5820035

####Check missing values

In [9]:
# Count null values in each columns 
print('Number of null values in creator_name: ',df.filter(df['creator_name'].isNull()).count())
print('Number of null values in userid: ',df.filter(df['userid'].isNull()).count())
print('Number of null values in comment: ',df.filter(df['comment'].isNull()).count())

Number of null values in creator_name: 32050
Number of null values in userid: 565
Number of null values in comment: 1051

####Drop data with no comments

In [11]:
df = df.na.drop(subset=["comment"])
print('Number of rows after droping null comment:',df.count())

Number of rows after droping null comment: 5818984

####Convert comment text to lower case

In [13]:
import pyspark.sql.functions as F
df_clean = df.withColumn('comment', F.lower(F.col('comment')))

####Data overview

In [15]:
df_clean.show()

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| i shared this to ...|
 Doug The Pug| 87.0| super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| tenho uma jiboia ...|
 ojatro|1031.0| i wanna see what ...|
 Tingle Triggers|1212.0| well shit now im ...|
Hope For Paws - O...|1806.0| when i saw the en...|
Hope For Paws - O...|2036.0| holy crap. that i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| call the teddy larry|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟|
Hope For Paws - O...|2911.0| that mother cat l...|
Hope For Paws - O...|2911.0| its people like h...|
 Talking Kitty Cat|2911.0| steve: no wet foo...|
 Brave Wilderness|3224.0| dont call this a ...|
 MaxluvsMya|3267.0| why are you alway...|
Rise Up Society F...|3372.0| deb tucker|
 The Dodo|3466.0| thats a deer isnt...|
 Brave Wilderness|3466.0| there is no safe ...|
 Brave Wilderness|3466.0| red before yellow...|
+--------------------+------+-------------------------------------+
only showing top 20 rows

#Part 1. Data preprocessing

### Tokenize the text data and create word2Vec features

Select 2MM data for this project to use.

In [19]:
from pyspark.sql.functions import rand 

df_clean.orderBy(rand(seed=0)).createOrReplaceTempView("table")
df_clean = spark.sql("select * from table limit 2000000")

In [20]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")
word2Vec = Word2Vec(inputCol="words", outputCol="features") #the tokenized feature captures semantic similarity of that word to other words

In [21]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_clean)
dataset = pipelineFit.transform(df_clean)

In [22]:
dataset.show(10)

+--------------------+---------+--------------------+--------------------+--------------------+
 creator_name| userid| comment| words| features|
+--------------------+---------+--------------------+--------------------+--------------------+
 Gohan The Husky| 106460.0|keep him! and cal...|[keep, him, and, ...|[-0.0242814878718...|
 Real Shock|1050336.0|я на все правильн...| []| (100,[],[])|
 Brian Barczyk| 636384.0| hogwarts 🐍🐍🐍🐍🐍| [hogwarts]|[0.11475221812725...|
 Think Like A Horse|1411449.0|i love how you al...|[i, love, how, yo...|[0.06695287060990...|
 Mạnh CFM| 879606.0| hai vai a manh| [hai, vai, a, manh]|[-0.0317196808755...|
 Home Aquatics Hobby|1995671.0| imma try this!| [imma, try, this]|[0.15546574940284...|
 Sad Cat|2039444.0|the 2nd dog was t...|[the, 2nd, dog, w...|[0.12009738194935...|
Hope For Paws - O...| 565118.0|my eyes on tears ...|[my, eyes, on, te...|[0.04250455430398...|
 FROSTY Life| 892104.0|nature protection...|[nature, protecti...|[0.01910656044880...|
 ViralHog|2270703.0| damn she thick| [damn, she, thick]|[0.03960126638412...|
+--------------------+---------+--------------------+--------------------+--------------------+
only showing top 10 rows

###Selecting 1000000 data to build the classifier

In [24]:
from pyspark.sql.functions import rand 

dataset.orderBy(rand(seed=0)).createOrReplaceTempView("table")
sampled = spark.sql("select * from table limit 1000000")

#### Label the data
This is an unlabeled dataset and we want to train a clasifier to identify cat and dog owners. Thus first thing to do is to label each comment. <br>
Label comment when he/she has dogs or cats. <br>
label comment when he/she don't have a dog or cat. <br>
Combine 1 and 2 as our training dataset, and rest of the dataset will be the data we predict. <br>

In [26]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

sampled = sampled.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                            .when(col("comment").like("%my dogs%"), 1) \
                           .when(col("comment").like("%i have a dog%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%my cats%"), 1) \
                           .when(col("comment").like("%i have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%i have a puppy%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                            .when(col("comment").like("%i have a pup%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my kitten%"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .when(col("comment").like("i have a kitty"),1)\
                           .when(col("comment").like("i have a kitten"),1)\
                           .when(col("comment").like("my own"),1)\
                           .otherwise(0)))

In [27]:
sampled.show(10)

+-------------------+---------+--------------------+--------------------+--------------------+-----+
 creator_name| userid| comment| words| features|label|
+-------------------+---------+--------------------+--------------------+--------------------+-----+
 Brave Wilderness| 254809.0|for as much as i ...|[for, as, much, a...|[-0.1172484595860...| 0|
 ТОП САМЫХ|1652025.0| л| []| (100,[],[])| 0|
 HammyLux| 159772.0|ah! your petco is...|[ah, your, petco,...|[0.08070177207183...| 0|
 LightningLpsTV|1881033.0|fun fact:hot dogs...|[fun, fact, hot, ...|[0.03406326798722...| 0|
 Mr. Max T.V.|1936350.0|thanks max i am a...|[thanks, max, i, ...|[-0.0309817560094...| 0|
 Think Like A Horse|2007426.0|u r soon right. h...|[u, r, soon, righ...|[0.08446431501458...| 0|
 The Pet Collective|1382459.0|*i love how the t...|[i, love, how, th...|[0.01836268778424...| 0|
 Viral Maniacs|1817340.0|лайк если завараж...| []| (100,[],[])| 0|
Keedes channel LIVE| 723319.0|15:16 ок а то я у...| [15, 16]|[0.03535630740225...| 0|
 Brave Wilderness|2092494.0|sillybilly thats ...|[sillybilly, that...|[-0.0838636914268...| 0|
+-------------------+---------+--------------------+--------------------+--------------------+-----+
only showing top 10 rows

### Downsampling the dataset

Note that number of negative labels is around 100 times more than positive labels, so here we need to downsampling the negative labels. By rule of thumb, the gap should be no more than 10 times. But here I make them balance to the ratio aroudn 1:2 (1 for positive: 2 for negative)

In [30]:
own_pets = sampled.filter(col('label')==1)
no_pets = sampled.filter(col('label')==0)
print("Number of confirmed users who own dogs or cats in training data set: ", own_pets.count())
print("Number of confirmed users who don't have pet's in training data set: ", no_pets.count())

Number of confirmed users who own dogs or cats in training data set: 9487
Number of confirmed users who don't have pet's in training data set: 990513

In [31]:
from pyspark.sql.functions import rand 
no_pets.orderBy(rand()).createOrReplaceTempView("table")

Num_Pos_Label = own_pets.count() 
Num_Neg_Label = no_pets.count()

df_no_pets_down = spark.sql("select * from table where limit {}".format(Num_Pos_Label*2))


In [32]:
print('Now after balancing the lables, we have ')   
print('Positive label: ', Num_Pos_Label)
print('Negtive label: ', df_no_pets_down.count())

Now after balancing the lables, we have 
Positive label: 9487
Negtive label: 18974

In [33]:
df_model = own_pets.union(df_no_pets_down)
df_model.show(10)

+--------------------+---------+--------------------+--------------------+--------------------+-----+
 creator_name| userid| comment| words| features|label|
+--------------------+---------+--------------------+--------------------+--------------------+-----+
Dog Training by K...| 14593.0|i have a chow. iv...|[i, have, a, chow...|[0.07402610394065...| 1|
 Big Cat Rescue|1970394.0|if my cat sounds ...|[if, my, cat, sou...|[0.08496134922218...| 1|
 meow meow|1704771.0|my cats are going...|[my, cats, are, g...|[0.07563407280083...| 1|
 The Dodo|1727704.0|my cat would not ...|[my, cat, would, ...|[0.11237937398254...| 1|
Zak Georges Dog T...|1217044.0|ok so i love your...|[ok, so, i, love,...|[0.05536452893766...| 1|
Hope For Paws - O...|1585600.0|should have calle...|[should, have, ca...|[-0.0310175169089...| 1|
 Vet Ranch|1412999.0|omg...i want her....|[omg, i, want, he...|[0.04523689354149...| 1|
Zak Georges Dog T...|1813543.0|my dog is not rea...|[my, dog, is, not...|[0.07030598035524...| 1|
 Robin Seplut|1521289.0|thanks for the in...|[thanks, for, the...|[0.03279427839443...| 1|
 Brian Barczyk| 869161.0|plz do not cry i ...|[plz, do, not, cr...|[0.02789124846458...| 1|
+--------------------+---------+--------------------+--------------------+--------------------+-----+
only showing top 10 rows

#Part 2. Model Training and Evaluation

In [35]:
train, test = df_model.randomSplit([0.8, 0.2], seed=12345)

###Logistic Regression

In [37]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

lr = LogisticRegression(featuresCol="features",labelCol="label" , maxIter=10, regParam=0.1, elasticNetParam=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
lrModel = lr.fit(train)

In [38]:
summary_lr = lrModel.summary
# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
import seaborn as sns
import matplotlib.pyplot as plt
roc = summary_lr.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.plot([0, 1], [0, 1], 'k--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve')
plt.show()

In [39]:
print("areaUnderROC: " + str(summary_lr.areaUnderROC))

areaUnderROC: 0.8849961224029992

In [40]:
# Set the model threshold to maximize F-Measure
fMeasure = summary_lr .fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
                .select('threshold').head()['threshold']
# Make predictions on test data. model is the model with combination of parameters
# that performed best.
predictions_lr = lrModel.transform(test,{lrModel.threshold: bestThreshold})
predictions_lr.show(10)

+--------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment| words| features|label| rawPrediction| probability|prediction|
+--------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 null| 939558.0|if i saw my cat f...|[if, i, saw, my, ...|[0.05614113026800...| 1|[0.4870847760543,...|[0.61941944002529...| 1.0|
 null|1265524.0|my cat would slau...|[my, cat, would, ...|[0.03301996208028...| 1|[0.01956351767371...|[0.50489072343337...| 1.0|
 null|1591257.0|notice the tiger ...|[notice, the, tig...|[0.10342899509878...| 1|[0.49246764454753...|[0.62068757380402...| 1.0|
 101rabbits| 946174.0|i used to have an...|[i, used, to, hav...|[0.01570107034907...| 1|[0.48477938070999...|[0.61887581901535...| 1.0|
 101rabbits|1465076.0|mocha has the sam...|[mocha, has, the,...|[-0.0828659053659...| 1|[-0.1071429124634...|[0.47323986663140...| 1.0|
 101rabbits|2069730.0|omg your bunnys a...|[omg, your, bunny...|[0.04464152043219...| 1|[0.42127325559381...|[0.60378788856477...| 1.0|
Aarons Animals| 937383.0|this is like my c...|[this, is, like, ...|[-0.0294186978672...| 1|[0.49364340508288...|[0.62096434910277...| 1.0|
Aarons Animals|1345702.0|i didnt think you...|[i, didnt, think,...|[0.18048949539661...| 1|[0.45657514529346...|[0.61220139160613...| 1.0|
Aarons Animals|1483408.0|my cat also did m...|[my, cat, also, d...|[0.05328114432367...| 1|[0.38110299468359...|[0.59413910427085...| 1.0|
Aarons Animals|2032829.0|my cat just died ...|[my, cat, just, d...|[-0.0069224577488...| 1|[0.42237302555556...|[0.60405095436119...| 1.0|
+--------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


def get_evaluation_result(predictions):
  evaluator = BinaryClassificationEvaluator(
      labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
  AUC = evaluator.evaluate(predictions)

  TP = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 1.0)].count()
  FP = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 1.0)].count()
  TN = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 0.0)].count()
  FN = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 0.0)].count()

  accuracy = (TP + TN)*1.0 / (TP + FP + TN + FN)
  precision = TP*1.0 / (TP + FP)
  recall = TP*1.0 / (TP + FN)


  print ("True Positives:", TP)
  print ("False Positives:", FP)
  print ("True Negatives:", TN)
  print ("False Negatives:", FN)
  print ("Test Accuracy:", accuracy)
  print ("Test Precision:", precision)
  print ("Test Recall:", recall)
  print ("Test AUC of ROC:", AUC)

print("Prediction result summary for Logistic Regression Model:  ")
get_evaluation_result(predictions_lr)

Prediction result summary for Logistic Regression Model: 
True Positives: 1601
False Positives: 858
True Negatives: 3020
False Negatives: 279
Test Accuracy: 0.8025356026398055
Test Precision: 0.651077673851159
Test Recall: 0.8515957446808511
Test AUC of ROC: 0.8771889445927803

###Random Forest

In [43]:

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# Train a RandomForest model.
rfModel = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=15)

# Train model.  This also runs the indexers.
model = rfModel.fit(train)

# Make predictions.
predictions_rf = model.transform(test)

# Select example rows to display.
predictions_rf.show(10)

+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment| words| features|label| rawPrediction| probability|prediction|
+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 null|1855339.0|i would protect m...|[i, would, protec...|[0.04566170562793...| 1|[4.33430724161909...|[0.28895381610793...| 1.0|
 101rabbits| 367209.0|hi im moving and ...|[hi, im, moving, ...|[8.29553438557518...| 1|[3.79561809136782...|[0.25304120609118...| 1.0|
 101rabbits|1018999.0|when i saw the na...|[when, i, saw, th...|[0.02316679899452...| 1|[5.91471206295764...|[0.39431413753050...| 1.0|
2CAN.TV - Ripley ...|1384044.0|you put a banana ...|[you, put, a, ban...|[0.08089200304239...| 1|[11.8407586516321...|[0.78938391010880...| 0.0|
2CAN.TV - Ripley ...|2382136.0|do you spend a lo...|[do, you, spend, ...|[0.12231800931951...| 1|[6.31245551337581...|[0.42083036755838...| 1.0|
 76Highboy Reloading|2462842.0|my dog is a repub...|[my, dog, is, a, ...|[0.03120583518304...| 1|[8.65896404212307...|[0.57726426947487...| 0.0|
 Aarons Animals|2032829.0|my cat just died ...|[my, cat, just, d...|[-0.0069224577488...| 1|[4.67274230288730...|[0.31151615352582...| 1.0|
 Aarons Animals|2381933.0|tried putting a g...|[tried, putting, ...|[0.08143107344706...| 1|[5.20213777417998...|[0.34680918494533...| 1.0|
Abandoned NorthJe...| 171330.0|awww so cute look...|[awww, so, cute, ...|[0.01436597201973...| 1|[8.02853183445600...|[0.53523545563040...| 0.0|
 Alex Knappenberger| 430815.0| my dog is the same|[my, dog, is, the...|[-0.1069111421704...| 1|[4.08098716593775...|[0.27206581106251...| 1.0|
+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

In [44]:
print("Prediction result summary for Random Forest Model:  ")
get_evaluation_result(predictions_rf)

Prediction result summary for Random Forest Model: 
True Positives: 1296
False Positives: 168
True Negatives: 3659
False Negatives: 619
Test Accuracy: 0.8629397422500871
Test Precision: 0.8852459016393442
Test Recall: 0.6767624020887728
Test AUC of ROC: 0.9416125621576833

###Gradient Boosted

In [46]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Train a GDBT model.
gbtModel = GBTClassifier(featuresCol="features",labelCol="label", maxIter=10)

# Train model.  This also runs the indexer.
model = gbtModel.fit(train)

# Make predictions.
predictions_gbt = model.transform(test)

# Select example rows to display.
predictions_gbt.show(10)


+-------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment| words| features|label| rawPrediction| probability|prediction|
+-------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 null|1859003.0|that guy who was ...|[that, guy, who, ...|[0.11277555616661...| 1|[-1.0628234487364...|[0.10662895531310...| 1.0|
 101rabbits| 367209.0|hi im moving and ...|[hi, im, moving, ...|[8.29553438557518...| 1|[-1.0001455315652...|[0.11917236564742...| 1.0|
 101rabbits| 946174.0|i used to have an...|[i, used, to, hav...|[0.01570107034907...| 1|[-1.0578667860207...|[0.10757698014139...| 1.0|
 Aaron Rift|1217127.0|wish my cat did t...|[wish, my, cat, d...|[0.05897291228175...| 1|[-0.9495973448550...|[0.13019964674365...| 1.0|
 Aarons Animals| 26421.0|best invasion eve...|[best, invasion, ...|[0.01281318785622...| 1|[-0.9382359811825...|[0.13279463704396...| 1.0|
 Aarons Animals| 99588.0|your videos are s...|[your, videos, ar...|[0.03866720924162...| 1|[0.78163211631509...|[0.82682125024673...| 0.0|
 Aarons Animals|1992132.0|my cat name is ma...|[my, cat, name, i...|[-0.0706151963211...| 1|[-0.0872436913765...|[0.45648849372193...| 1.0|
AleiaAnimalLover368|1129471.0|im so sorry realy...|[im, so, sorry, r...|[0.02153120976355...| 1|[-0.8601617163336...|[0.15182950817778...| 1.0|
 Alex Knappenberger| 43143.0|if i had to guess...|[if, i, had, to, ...|[0.04421059365995...| 1|[-0.2955057156273...|[0.35640281867581...| 1.0|
 Alex Knappenberger| 695428.0|i hate this forci...|[i, hate, this, f...|[0.14996320175889...| 1|[-0.6290956747563...|[0.22128539844957...| 1.0|
+-------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

In [47]:
print("Prediction result summary for Gradient Boosted Model:  ")
get_evaluation_result(predictions_gbt)

Prediction result summary for Gradient Boosted Model: 
True Positives: 1555
False Positives: 362
True Negatives: 3480
False Negatives: 357
Test Accuracy: 0.8750434480361488
Test Precision: 0.8111632759520083
Test Recall: 0.8132845188284519
Test AUC of ROC: 0.9399335034226104

#Part 3. Hyperparamter Tuning

#### Tune hyperameters for GDBT model

In [50]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

#Tune parameter using grid search and cross validation
paramGrid_gbt = ParamGridBuilder()\
               .addGrid(rfModel.numTrees, [int(x) for x in np.linspace(start = 10, stop = 50, num = 3)]) \
               .addGrid(rfModel.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
               .build()

evaluator=BinaryClassificationEvaluator()

crossval_gbt = CrossValidator(
               estimator=gbtModel,
               estimatorParamMaps=paramGrid_gbt,
               evaluator=evaluator,
               numFolds=3)

####Get the best model with the best parameters

In [52]:
#Best model with tuned parameters
cvModel_gbt = crossval_gbt.fit(train)

# Make predictions.
predictions = cvModel_gbt.transform(test)

# Select example rows to display.
predictions.show(10)

print("Prediction result summary on test data for Gradient Boosted Model with tuned parameters:  ")
get_evaluation_result(predictions)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment| words| features|label| rawPrediction| probability|prediction|
+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 null|1596933.0|my doggo freind m...|[my, doggo, frein...|[0.09965891477673...| 1|[0.82813834859425...|[0.83973756056947...| 0.0|
 null|2016303.0|i cant have a bir...|[i, cant, have, a...|[0.11987633439194...| 1|[-0.6586928883918...|[0.21125356090386...| 1.0|
 null|2381008.0|i see your snake ...|[i, see, your, sn...|[0.07449154711018...| 1|[-0.4705064751925...|[0.28069577674430...| 1.0|
 1BubbaMike1| 531433.0|my cat thinks dat...|[my, cat, thinks,...|[0.02003000473932...| 1|[-1.0093993512012...|[0.11724326475488...| 1.0|
 2 Pitbulls| 358681.0|my dog loves to s...|[my, dog, loves, ...|[0.07868712005967...| 1|[-0.4418116413799...|[0.29242750875345...| 1.0|
2CAN.TV - Ripley ...| 662264.0|i can assure you ...|[i, can, assure, ...|[0.06884804462242...| 1|[-0.7905345086907...|[0.17064413636959...| 1.0|
 AfroHerpkeeper| 839735.0|i might be gettin...|[i, might, be, ge...|[0.08326297533725...| 1|[-0.9223485334762...|[0.13649672519659...| 1.0|
 Alex Knappenberger| 415413.0|you want to see a...|[you, want, to, s...|[0.15721849208189...| 1|[-0.7469945237502...|[0.18332374798336...| 1.0|
 Alex Knappenberger| 442327.0|my dog has the si...|[my, dog, has, th...|[0.04860801990809...| 1|[-0.7271768059751...|[0.18933244502029...| 1.0|
 Alex Knappenberger|1586986.0|my dog does that ...|[my, dog, does, t...|[-0.0051822586517...| 1|[-0.2425860659383...|[0.38103154766661...| 1.0|
+--------------------+---------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

Prediction result summary on test data for Gradient Boosted Model with tuned parameters: 
True Positives: 1599
False Positives: 390
True Negatives: 3434
False Negatives: 341
Test Accuracy: 0.8731783483691881
Test Precision: 0.803921568627451
Test Recall: 0.8242268041237113
Test AUC of ROC: 0.947897404402701

In [53]:
#Fit the model to the full dataset and evaluate
predictions_full = cvModel_gbt.transform(df_model)
print("Prediction result summary on full dataset for Gradient Boosted Model with tuned parameters:  ")
get_evaluation_result(predictions_full)

Prediction result summary on full dataset for Gradient Boosted Model with tuned parameters: 
True Positives: 7768
False Positives: 1934
True Negatives: 17040
False Negatives: 1719
Test Accuracy: 0.8716489230877341
Test Precision: 0.8006596578025149
Test Recall: 0.8188046800885422
Test AUC of ROC: 0.9460317975025871

In [54]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import itertools

#Define a function print the confusion matraix
def plot_confusion_matrix(predictions,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    y_true =predictions.select("label")
    y_true = y_true.toPandas()
    y_pred = predictions.select("prediction")
    y_pred = y_pred.toPandas()
    class_temp = predictions.select("label").groupBy("label")\
                        .count().sort('count', ascending=False).toPandas()
    class_names = class_temp["label"].values.tolist()
    
    cm = confusion_matrix(y_true, y_pred, labels = class_names)
   
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [55]:
plot_confusion_matrix(predictions_full)

We will use to cvModel_gbt as our classifier to classify all the users and make analysis based on the result.

#Part 4. Model Application

####Classify the full user data to identify pet owners

In [59]:
predictions= cvModel_gbt.transform(dataset)
pet_owners = predictions.filter(col('prediction')==1.0)
pet_owners.show(10)

+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name| userid| comment| words| features| rawPrediction| probability|prediction|
+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 Think Like A Horse|1411449.0|i love how you al...|[i, love, how, yo...|[0.06695287060990...|[-0.2880773068126...|[0.35981789616864...| 1.0|
Home Aquatics Hobby|1995671.0| imma try this!| [imma, try, this]|[0.15546574940284...|[-0.0101956099006...|[0.49490237168207...| 1.0|
 Sad Cat|2039444.0|the 2nd dog was t...|[the, 2nd, dog, w...|[0.12009738194935...|[-0.2941174812419...|[0.35703993827053...| 1.0|
 Kaimuki Backyard| 550340.0|my guppy fries di...|[my, guppy, fries...|[0.03537533077178...|[-0.7689240750038...|[0.17684830820772...| 1.0|
 Brave Wilderness| 385817.0|*a year later he ...|[a, year, later, ...|[0.08092219396672...|[-0.0734222311071...|[0.46335471022914...| 1.0|
 VISUALEYESFILMS| 967441.0|exactly luv hit a...|[exactly, luv, hi...|[-0.0230391457676...|[-0.0737382573100...|[0.46319754954185...| 1.0|
 The Dodo| 473941.0|i found a mouse i...|[i, found, a, mou...|[0.04896848976251...|[-0.7282867922673...|[0.18899194601315...| 1.0|
 TheMeanKitty|2477782.0|lol!!!  oh my wor...|[lol, oh, my, wor...|[-0.0370392208232...|[-0.5307147081293...|[0.25703638616869...| 1.0|
 Big Cat Rescue|2448356.0| kitty!| [kitty]|[0.25328260660171...|[-0.6530059269260...|[0.21315497050281...| 1.0|
 The Dodo|2469782.0|i have raised sev...|[i, have, raised,...|[0.04547544250936...|[-0.4400376707147...|[0.29316216655448...| 1.0|
+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

####Transform the full user data to TF-IDF matrix
In order to build a LDA model to analyze the topic of pet owners, we used to tf-idf instead of word2vec to transform the user comments feature, because tf-idf will assign unequal weights to words within documents. <br>
The intuition behind it is that if a word occurs multiple times in a document, we should boost its relevance as it should be more meaningful than other words that appear fewer times (TF). At the same time, if a word occurs many times in a document but also along many other documents, maybe it is because this word is just a frequent word; not because it was relevant or meaningful (IDF).

In [61]:
#Define a function to preprocess the text
#Remove stopwords

import re as re
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.feature import StopWordsRemover

def text_process(df):
  # Define a list of stop words or use default list
  remover = StopWordsRemover()
  Stopwords = remover.getStopWords() 
  remover.setInputCol("words")
  remover.setOutputCol("words_no_stopw")
  
  alpha_udf = udf(lambda tokens: [token for token in tokens if token.isalpha()]) 
  length_udf = udf(lambda tokens: [token for token in tokens if len(token)>2],ArrayType(StringType()))
  
  df = remover.transform(df) #remove stop words
  #df = df.withColumn("words_no_stopw",alpha_udf("words_no_stopw"))   #check if they’re alpha numeric
  df = df.withColumn("words_no_stopw",length_udf("words_no_stopw"))  #remove any words or typos which are less than two letters
  return df

In [62]:
pet_owners = text_process(pet_owners)

pet_owners.show(10)

+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
 creator_name| userid| comment| words| features| rawPrediction| probability|prediction| words_no_stopw|
+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
 Think Like A Horse|1411449.0|i love how you al...|[i, love, how, yo...|[0.06695287060990...|[-0.2880773068126...|[0.35981789616864...| 1.0|[love, always, sa...|
Home Aquatics Hobby|1995671.0| imma try this!| [imma, try, this]|[0.15546574940284...|[-0.0101956099006...|[0.49490237168207...| 1.0| [imma, try]|
 Sad Cat|2039444.0|the 2nd dog was t...|[the, 2nd, dog, w...|[0.12009738194935...|[-0.2941174812419...|[0.35703993827053...| 1.0|[2nd, dog, scooby...|
 Kaimuki Backyard| 550340.0|my guppy fries di...|[my, guppy, fries...|[0.03537533077178...|[-0.7689240750038...|[0.17684830820772...| 1.0|[guppy, fries, di...|
 Brave Wilderness| 385817.0|*a year later he ...|[a, year, later, ...|[0.08092219396672...|[-0.0734222311071...|[0.46335471022914...| 1.0|[year, later, plate]|
 VISUALEYESFILMS| 967441.0|exactly luv hit a...|[exactly, luv, hi...|[-0.0230391457676...|[-0.0737382573100...|[0.46319754954185...| 1.0|[exactly, luv, hi...|
 The Dodo| 473941.0|i found a mouse i...|[i, found, a, mou...|[0.04896848976251...|[-0.7282867922673...|[0.18899194601315...| 1.0|[found, mouse, ba...|
 TheMeanKitty|2477782.0|lol!!!  oh my wor...|[lol, oh, my, wor...|[-0.0370392208232...|[-0.5307147081293...|[0.25703638616869...| 1.0|[lol, word, cute,...|
 Big Cat Rescue|2448356.0| kitty!| [kitty]|[0.25328260660171...|[-0.6530059269260...|[0.21315497050281...| 1.0| [kitty]|
 The Dodo|2469782.0|i have raised sev...|[i, have, raised,...|[0.04547544250936...|[-0.4400376707147...|[0.29316216655448...| 1.0|[raised, several,...|
+-------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
only showing top 10 rows

Transform the daraCountVectorizer takes this data and returns a sparse matrix of term frequencies attached to the original Dataframe. Same thing goes for the IDF.

In [64]:
pet_owners = pet_owners.select("words_no_stopw")

In [65]:
# TF
cv = CountVectorizer(inputCol="words_no_stopw", 
                     outputCol="raw_features",  
                     vocabSize = 5000,
                     minDF=5.0) # the minimum number (or fraction if < 1.0) of documents a term must appear in to be included in the vocabulary.

cvmodel = cv.fit(pet_owners)
result_cv = cvmodel.transform(pet_owners)
result_cv.show()
print(len(cvmodel.vocabulary))  # vocabulary size 

+--------------------+--------------------+
 words_no_stopw| raw_features|
+--------------------+--------------------+
[love, always, sa...|(5000,[2,19,32,73...|
 [imma, try]|(5000,[137,2154],...|
[2nd, dog, scooby...|(5000,[1,28,161,2...|
[guppy, fries, di...|(5000,[67,87,227,...|
[year, later, plate]|(5000,[75,345,212...|
[exactly, luv, hi...|(5000,[323,393,22...|
[found, mouse, ba...|(5000,[8,31,47,59...|
[lol, word, cute,...|(5000,[2,19,25,35...|
 [kitty]| (5000,[142],[1.0])|
[raised, several,...|(5000,[5,38,67,87...|
[rainy, gloomy, d...|(5000,[0,1,29,33,...|
[thumbnail, thoug...|(5000,[5,19,26,11...|
[bruh, ended, fck...|(5000,[846,2132],...|
[story, continues...|(5000,[3,19,33,49...|
[love, dogs, fine...|(5000,[2,14,344,8...|
[omg, got, new, p...|(5000,[8,33,76,82...|
[catnip, basicall...|(5000,[10,792,970...|
[mansplaining, la...|(5000,[13,67,418]...|
[wow, sister, sai...|(5000,[5,12,18,24...|
[taxonomist, hopp...|(5000,[63,74,87,1...|
+--------------------+--------------------+
only showing top 20 rows

5000

In [66]:
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 
result_tfidf.show()

+--------------------+--------------------+--------------------+
 words_no_stopw| raw_features| features|
+--------------------+--------------------+--------------------+
[love, always, sa...|(5000,[2,19,32,73...|(5000,[2,19,32,73...|
 [imma, try]|(5000,[137,2154],...|(5000,[137,2154],...|
[2nd, dog, scooby...|(5000,[1,28,161,2...|(5000,[1,28,161,2...|
[guppy, fries, di...|(5000,[67,87,227,...|(5000,[67,87,227,...|
[year, later, plate]|(5000,[75,345,212...|(5000,[75,345,212...|
[exactly, luv, hi...|(5000,[323,393,22...|(5000,[323,393,22...|
[found, mouse, ba...|(5000,[8,31,47,59...|(5000,[8,31,47,59...|
[lol, word, cute,...|(5000,[2,19,25,35...|(5000,[2,19,25,35...|
 [kitty]| (5000,[142],[1.0])|(5000,[142],[4.30...|
[raised, several,...|(5000,[5,38,67,87...|(5000,[5,38,67,87...|
[rainy, gloomy, d...|(5000,[0,1,29,33,...|(5000,[0,1,29,33,...|
[thumbnail, thoug...|(5000,[5,19,26,11...|(5000,[5,19,26,11...|
[bruh, ended, fck...|(5000,[846,2132],...|(5000,[846,2132],...|
[story, continues...|(5000,[3,19,33,49...|(5000,[3,19,33,49...|
[love, dogs, fine...|(5000,[2,14,344,8...|(5000,[2,14,344,8...|
[omg, got, new, p...|(5000,[8,33,76,82...|(5000,[8,33,76,82...|
[catnip, basicall...|(5000,[10,792,970...|(5000,[10,792,970...|
[mansplaining, la...|(5000,[13,67,418]...|(5000,[13,67,418]...|
[wow, sister, sai...|(5000,[5,12,18,24...|(5000,[5,12,18,24...|
[taxonomist, hopp...|(5000,[63,74,87,1...|(5000,[63,74,87,1...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

####Build a LDA model - topic modeling

##### Why LDA?
LDA trades off two “conflicting” goals: <br>
1: For each document, allocate its words to as few topics aspossible. <br>
2: For each topic, assign high probability to as few terms aspossible. <br>
LDA is a probabilistic model of text used to find topics thatdescribe a corpus.<br>
LDA casts the problem of discovering themes in largedocument collections as a posterior inference problem.LDA lets us visualize the hidden thematic structure in largecollections, and generalize new data to fit into that structure.

In [69]:

from pyspark.ml.clustering import LDA
num_topics = 8
lda = LDA(k = num_topics,
          maxIter = 50 #number of iterations
          )

ldaModel = lda.fit(result_tfidf)

#####Print the top 20 words in each topic

In [71]:
# Print topics and top-weighted terms
topics = ldaModel.describeTopics(maxTermsPerTopic=20) #return the index of words from dictionary created by CountVector
vocabArray = cvmodel.vocabulary

ListOfIndexToWords = udf(lambda wl: list([vocabArray[w] for w in wl]))
#FormatNumbers = udf(lambda nl: ["{:1.4f}".format(x) for x in nl])

topics_words = topics.select(ListOfIndexToWords(topics.termIndices).alias('words'))
topics_words.show(truncate=False, n=num_topics)

+--------------------------------------------------------------------------------------------------------------------------------------------+
words |
+--------------------------------------------------------------------------------------------------------------------------------------------+
[cry, one, crying, like, cat, dog, made, little, dude, got, lol, love, aww, lost, back, haha, hungry, really, time, weird] |
[kitty, fish, black, like, one, something, know, tank, cat, dont, heart, think, dog, someone, leg, well, get, got, done, time] |
[love, channel, videos, video, animals, happy, much, one, like, watching, really, dog, get, year, great, day, birthday, always, watch, well]|
[cats, cat, like, looks, kittens, cute, lol, kitten, love, hair, adorable, one, bird, food, want, dog, black, robin, get, adopt] |
[dog, husky, dogs, like, puppy, omg, rabbit, gohan, cute, love, cried, want, max, bunny, wants, wish, mom, get, shelby, one] |
[male, female, like, dog, house, bed, sleep, one, dont, time, also, know, get, dogs, got, around, run, birds, never, put] |
[horse, dog, horses, german, like, get, hamster, got, dont, bit, ride, riding, said, shepherd, back, think, fuck, kid, never, one] |
[people, know, dont, help, mother, get, care, like, animals, one, anyone, didnt, vet, want, dogs, else, love, really, dog, puppies] |
+--------------------------------------------------------------------------------------------------------------------------------------------+

#####Visualize the wordcloud of each topic

In [73]:
!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#Define a function to draw wordclouds of topics

def wordcloud(topics):
  wordcloud = []
  for i in range(len(topics)):
    text = str(topics.iloc[i,0])
    #text = " ".join([(k + " ")*v for k,v in topics.iloc[i].items()])
    #temp_cloud = WordCloud().generate(text)
    temp_cloud = WordCloud(background_color="white", max_words=10000, collocations = False,
               contour_width=3, contour_color='steelblue',max_font_size=40)
    temp_cloud = temp_cloud.generate(text) # Generate a word cloud image
    if len(wordcloud) == 0:
      wordcloud = [temp_cloud]
    else:
      wordcloud.append(temp_cloud)
  # Display the generated image:
  # the matplotlib way:
  w=10
  h=10
  fig=plt.figure(figsize=(40,10))
  columns = 4
  rows = 2
  for i in range(1, columns*rows +1):
      img = wordcloud [i-1]
      fig.add_subplot(rows, columns, i)
      plt.imshow(img)
      plt.axis("off")    
  plt.show()
  
 

In [74]:
wordcloud(topics_pdf)

#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [76]:
#Identify cat and dog owners
predictions= cvModel_gbt.transform(dataset)
pet_owner= predictions.filter(col('prediction')==1.0).select('creator_name','userid','comment')
pet_owner.show(truncate=False)

+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
creator_name |userid |comment |
+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Think Like A Horse |1411449.0|i love how you always say dummys or shut up lol makes me laugh |
Home Aquatics Hobby|1995671.0|imma try this! |
Sad Cat |2039444.0|the 2nd dog was the scooby doo breed. id never seen it before. |
Kaimuki Backyard |550340.0 |my guppy fries died 1 or 2 every week is it normal? |
Brave Wilderness |385817.0 |*a year later he is now on a plate. . . * |
VISUALEYESFILMS |967441.0 |exactly luv hit after hit |
The Dodo |473941.0 |i found a mouse in my basement once that was stuck to a sticky spider trap. i didnt have the heart to leave him there to die so i got my garden gloves and took the whole thing outside and got him unstuck which took some doing but he was ok. then i let him go over the fence in a field. |
TheMeanKitty |2477782.0|lol!!!  oh my word that is too cute & so precious!  i love her reaction!  her mind was totally blown! |
Big Cat Rescue |2448356.0|kitty! |
The Dodo |2469782.0|i have raised several squirrels. they are 1 person animals. i was devastated when they died- the oldest being 5 1/2. they are lovable intelligent and you get so attached. mine slept with me every night & went to work with my husband. |
Gohan The Husky |1416861.0|on a rainy gloomy day found this wonderful channel..specially because gohan looks like mishka ..... reminds me of her... seriously guy u have a wonderful dog. keep making vedios for us |
Solid Gold Aquatics|1672859.0|the thumbnail though lol also how did they get into your aquarium? |
Chriskillergaming |2485163.0|bruh you ended my fcking streak 😭 |
Brave Wilderness |2044294.0|the story continues....shortly after they have released the velvet worm.....worm: folks! there is something - there was this giant eye watching me!worms: lol!worm: seriously! it was a giant life form and it took me away!worms: sure and did they anal probe you? lmao!worm: bleh! you guys suck. one day youll realize that im right.worms: take less drugs and do you work.|
Brian Barczyk |233974.0 |i love dogs a i am fine you showing your emissions |
stacyvlogs |24582.0 |omg we got my new puppy ruby and she had kennel cofth the day be for we wint on vachion but we got her and she is the best |
Cole & Marmalade |334337.0 |catnip is basically weed for cats then... |
Paper Ghost |573364.0 |the mansplaining lady triggers me every time. every time. |
Brave Wilderness |1583886.0|wow my sister said i want to see him get bit! (she is only 4 years old) :) |
HoppingHammy |158503.0 |taxonomist and hoppinghammy you saved my hamsters life!! thank you!! he would have died if i couldnt have done this until the emergency vet arrived! |
+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [77]:
#Compute the fraction of cat or dog owners
owner_fr = round(pet_owner.count()/predictions.count()*100,2)
print(str(owner_fr)+ '% of the total users own cats or dogs in this Youtube comment dataset.')

10.74% of the total users own cats or dogs in this Youtube comment dataset.

#### 5. Analysis and Future work

Step 1: Identify Cat And Dog Owners
Find the users who are cat and/or dog owners.

Step 2: Build And Evaluate Classifiers
Build classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Classify All The Users
Apply the cat/dog classifiers to all the users in the dataset. Estimate the fraction of all users
who are cat/dog owners.

Step 4: Extract Insights About Cat And Dog Owners
Find topics important to cat and dog owners.

Step 5: Identify Creators With Cat And Dog Owners In The Audience
Find creators with the most cat and/or dog owners. Find creators with the highest statistically
significant percentages of cat and/or dog owners.

#Report
**Motivation:** My motivation of this project is because I myself own a cat, and I love watching cat related videos on Youtube. As I watch more and more videos, Youtube's algorithm was able to learn my video-watching habit and started to recommend more related content to me. But I feel like by identifying me as a cat owner, there can be more cool stuff to be done which is beyond the video recommendations. <br>

**Data and Method:** In this project, I used a sample of the Youtube comments text data to build a classifier to predict the pet and dog owners. Then, using the model trained, I identified those cat and dog owners in the full dataset. Then I built a LDA (Latent Dirichlet Allocation) model to classify the user's comments into 8 topics and plotted the wordcloud of each topic. <br>
For example, after recognizing the cat and dog owners, we can do in-depth analysis on these user's comments, then there will be changces that we gain valuable insights from those comments and find potential hot topics among this cohort. These findings can be applied to product design for pet services/product companies or pet video creators, because they can gain a good sense of what's been heavily discussed and what's trending right now, so that they can tailor their products/content to better target the specific group of users. <br>
#####Let's walk through how this project's been done!
**Step 1: Data Exploration and Cleaning**<br>
(1)There are 5,820,035 rows of user comments in total.Rows with null user id have been kept but those will null comments have been dropped. Comments are been converted to lowercase.<br> 
(2)Selected 2,000,000 rows of data to implement this whole project considering the time complexity. (Probably will run with the whole dataset again in the future)<br> 
(3)Tokenized the comment and created word2Vec features in order to train a classification model. <br> 
(4)Randomly sampled 1,000,000 rows of data to train the classifier and manually labeled the comments as 'has cat&dog' and 'no cat&dog'.<br> 
(5)This is a highly imbalanced dataset (with negative labels around 100 times more than positive labels), so I downsampled the negative labels (1 for positive:2 for negative).

**Step 2: Model Selection and Tuning**<br> 
(1)Splitted the data into training set and testing set.<br>
(2)Tained a classifier using three types of machine learning algorithm: logistic regression, random forest, and Gradient Boosting.<br>
(3)Utilized ROC/AUC, confusion matrix, accuracy, precision and recall as model evaluation metrics to evaluate the three models performance.<br>
(4)Selected Gradient Boosted algorithm to do hyparameter tuning since it has the best performance on the test data set.<br>
(5)Tuned the hyperparameters based on grid search and re-evaluated the model performace: Accuracy (0.87), Precision (0.8), Recall (0.82), AUV of ROC (0.94).

**Step 3: Model Application - LDA**<br>
(1)Applied the best GDBT model onto the full dataset and identified users who own dogs or cats. 10.74% of users own dogs or cats. <br>
(2)Preprocessed the dog&cat owners' text data to TF-IDF matrices and built a LDA model - set the topic number to 8. (This can be further improved on)<br>
(3)Got the LDA topic result and visualized the wordcloud of each topic based on word frequencied of each topic.

**Step 4: Insights gained from topic modeling**<br>
(1)Based on the wordcloud, the topic of each segment is not that easy to interpret, but I will try my best to garner as much information as possible based on my business sense.<br>
(2)Semantic meaning of each topic: Cat/Dog crying, kitty eating fish, I love waching this dog/animal Youtube channel, Cats are cute/adorable (chasing bird), Dog(husky) are cute (chasing rabbit?), gender differences for dogs, hourse related content, People care for dogs/animals.

**Recommendation based on the insights:**<br>
(1)Dog/cat related Youtube channels are super popular among dog/cat owners, this descripes their watching habit and Youtubers can post more videos to attract these users.<br>
(2)People love watching kitten eating fish/chasing bird, dogs (husky in particular) chaing rabbit, and content related to hourse riding. Youtubers can incorporate those content into their videos. <br>
(3)Following up on the above mentioned topics, ads for dog/cat competitions/social events can target this cohort, and horse-riding activity companies can see this cohort as their potential customers as well, since people watching those videos may develop an intention to engage in those activities in real life.<br>
(4)People are having discussions around pet genders and carings, it suggests that people with dog&cat pay more attention to animal care. Ads about pet commutiny (donation, adoption) can target those users.

**Area of Improvements of this project:**<br>
(1)Use the whole dataset may give better results in both models.<br>
(2)Find a better way to label the comment - this can be done by researching more into the pet owner behavior.<br>
(3)Get more user feature data other than user id, such as age, location, occupation, family size and some other behavioral features like Youtube watching record. - Based on those data, we can do user segmentation analysis before and after the classification as to get better results of identifying users and analyzing the owners' preferences.
(4)For the LDA part, we can calculate the model perplxity to select the optimal number of topic. We can also do more feature engineering to make the text data more ingestible.